In [31]:
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    def forward(self, input):
        # TODO: return output
        pass

    def backward(self, output_gradient, learning_rate):
        # TODO: update parameters and return input gradient
        pass

In [32]:
import numpy as np
from scipy import signal
from layer import Layer


#-- This is creating a  Convolutional layer
class Convolutional(Layer):
    def __init__(self, input_shape, kernel_size, depth):
        input_depth, input_height, input_width = input_shape
        self.depth = depth
        self.input_shape = input_shape
        self.input_depth = input_depth
        self.output_shape = (depth, input_height - kernel_size + 1, input_width - kernel_size + 1)
        self.kernels_shape = (depth, input_depth, kernel_size, kernel_size)
        self.kernels = np.random.randn(*self.kernels_shape)
        self.biases = np.random.randn(*self.output_shape)

#-- Craete a forward pass
    def forward(self, input):
        self.input = input
        self.output = None
        # TODO: Implement the forward method using the formula provided in the powerpoint. 
        for i in range(self.depth):
            for j in range(self.input_depth):
                self.ouput[i] += signal.correlate2d(self.input[j], self.kernels[i,j], "valid")
        # You may add or remove any variables that you wish. 
        return self.output

#-- Now work backrward to get the initial forward pass
    def backward(self, output_gradient, learning_rate):
        # TODO: initialize the kernels_gradient and input_gradient.
        kernels_gradient = np.zeros(self.kernels_shape)
        input_gradient = np.zeros(self.input_shape)

        # TODO: implement the back pass here. The equations in the ppt may help, but you're free to
        # add as much or as little code as you'd like. 

#-- This is to go throught the amount of layers
        for i in range(self.depth):
            for j in range(self.input_depth):
                kernels_gradient[i,j] = signal.correlate2d(self.input[j], output_gradient[i], "valid")
                input_gradient[j] += signal.convolve2d(output_gradient[i], self.kernels[i,j], "full")



        # TODO: update the kernels and biases
        self.kernels -= learning_rate * kernels_gradient
        self.biases -= learning_rate * output_gradient

        return input_gradient


In [33]:
import numpy as np
from layer import Layer

#-- We need to reshape here 
class Reshape(Layer):
    def __init__(self, input_shape, output_shape):
        self.input_shape = input_shape
        self.output_shape = output_shape

    def forward(self, input):
        # TODO: reshape the input to the output_shape and return it.
        return np.reshape(input, self.output_shape)

    def backward(self, output_gradient, learning_rate):
        # TODO: reshape the output to the input_shape and return it.
        #-- we do the reshape of backwards usuing output gradient since
        #-- we are working backwards
        return np.reshape(output_gradient, self.input_shape)


In [34]:
from turtle import window_height
import numpy as np
from layer import Layer

#-- Calculate dense with forward and backward's again
class Dense(Layer):
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(output_size, input_size)
        self.bias = np.random.randn(output_size, 1)

    def forward(self, input):
        # TODO: apply linear transformation to the input. see ppt for equation. 
        return np.dot(self.weights, self.input) + self.bias
        

    def backward(self, output_gradient, learning_rate):
        # TODO: update the weights and bias
        weights_gradient = np.dot(output_gradient, self.input.T)
        input_gradient = np.dot(self.weights.T, output_gradient)
        self.weights -= learning_rate * weights_gradient
        self.bias -= learning_rate * output_gradient
        return input_gradient



In [35]:
#-- since we are doing a binary classifcation it is justifiable
#-- to use bianry cross entropy loss
import numpy as np

#-- THis will calculate the mean 
def binary_cross_entropy(y_true, y_pred):
    # TODO: return the binary_cross_entropy loss
    return np.mean(-y_true * np.log(y_pred) - (1 - y_true) * np.log(1 - y_pred))

def binary_cross_entropy_prime(y_true, y_pred):
    # TODO: return the binary_cross_entropy_prime. 
    # Note, this is the formula on the bottom in the ppt slides.
    return ((1 - y_true) / (1 - y_pred) - y_true / y_pred) / np.size(y_true)


In [36]:
import numpy as np
from activation import Activation

#-- THis is for the activations 
class Tanh(Activation):
    ''' 
    Optional Tanh function if you'd like to try alternatives and see what happens.
    '''
    def __init__(self):
        def tanh(x):
            return np.tanh(x)

        def tanh_prime(x):
            return 1 - np.tanh(x) ** 2

        super().__init__(tanh, tanh_prime)

class Sigmoid(Activation):
    def __init__(self):
        def sigmoid(x):
            # TODO: return the sigmoid of x
            return 1 / (1 + np.exp(-x))

        def sigmoid_prime(x):
            # TODO: return the derivative
            s = sigmoid(x)
            return s * (1 - s)

        super().__init__(sigmoid, sigmoid_prime)




In [37]:
def predict(network, input):
    output = input
    for layer in network:
        output = layer.forward(output)
    return output

#-- This is to train the network and output an error
def train(network, loss, loss_prime, x_train, y_train, epochs = 1000, learning_rate = 0.01, verbose = True):
    for e in range(epochs):
        error = 0
        for x, y in zip(x_train, y_train):
            # forward
            output = predict(network, x)

            # error
            error += loss(y, output)

            # backward
            grad = loss_prime(y, output)
            for layer in reversed(network):
                grad = layer.backward(grad, learning_rate)

        error /= len(x_train)
        if verbose:
            print(f"{e + 1}/{epochs}, error={error}")

In [38]:
import numpy as np
from keras.datasets import mnist
from keras.utils import np_utils



#-- preproces the data 
def preprocess_data(x, y, limit):
    ''' 
    Will limit our data since using the whole thing will take forever on a cpu especially since we're
    implementing this from scratch.
    '''
    zero_index = np.where(y == 0)[0][:limit]
    one_index = np.where(y == 1)[0][:limit]
    all_indices = np.hstack((zero_index, one_index))
    all_indices = np.random.permutation(all_indices)
    x, y = x[all_indices], y[all_indices]
    x = x.reshape(len(x), 1, 28, 28)
    x = x.astype("float32") / 255
    y = np_utils.to_categorical(y)
    y = y.reshape(len(y), 2, 1)
    return x, y

# load MNIST from server, limit to 100 images per class since we're not training on GPU
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, y_train = preprocess_data(x_train, y_train, 100)
x_test, y_test = preprocess_data(x_test, y_test, 100)

#-- THis is our network
# TODO: Add our layers and the flow of input into this list. 
network = [
    
    Convolutional((1, 28, 28), 3, 5),
    Sigmoid(),
    Reshape((5, 26, 26), (5 * 26 * 26, 1)),
    Dense(5 * 26 * 26, 100),
    Sigmoid()
]

epochs = 10
learning_rate = 0.1
#
# train
for e in range(epochs):
    error = 0
    for x, y in zip(x_train, y_train):
        # forward
        output = x
        for layer in network:
            output = layer.forward(output)

        # TODO: update our error
        error += binary_cross_entropy(y, output)

        # TODO: perform back prop 
        grad = binary_cross_entropy_prime(y, output)
        for layer in reversed(network):
           grad = layer.backward(grad, learning_rate)

    error /= len(x_train)
    print(f"{e + 1}/{epochs}, error={error}")

# TODO: run the test data through and print out your predictions
for x, y in zip(x_test, y_test):
   output = x
   for layer in network:
     ouput = layer.forward(output)
     print(f"pred: {np.argmax(output)}, true: {np.argmax(y)}")


AttributeError: 'Convolutional' object has no attribute 'ouput'